In [ ]:
# Load packages
import numpy as np
import pandas as pd
import warnings, gc, sys, json
from m5_utils import *

pd.set_option('display.float_format', lambda x: '%.5f' % x)
pd.set_option('display.max_columns', 100)

In [ ]:
with open('input/dict_m5.json') as json_file:
    dict_m5 = json.load(json_file)
with open('input/dict_m5_inv.json') as json_file:
    dict_m5_inv = json.load(json_file)

In [ ]:
def convert_numbers(df, dict_m5_inv):
    map_year = { 2011:1, 2012:2, 2013:3, 2014:4, 2015:5, 2016:6}

    for col in df.columns:
        if col not in ['demand','wm_yr_wk','date','wday','month','year','snap_CA','snap_TX','snap_WI','sell_price']:
            col_dict = dict_m5_inv[col]
            df[col] = df[col].map(col_dict)
    for col in df.columns:
        if col  in ['year']:
            df[col] = df[col].map(map_year)
    
    df = reduce_mem_usage(df)
    
    return df

In [ ]:
%%time
train = pd.read_pickle('input/train_v0.pkl')
print(train.shape);train.head()

In [ ]:
# train['id'] = train['id'].transform(lambda x: x.replace('_evaluation','_validation'))
train = convert_numbers(train, dict_m5_inv)

In [ ]:
cat_cols = train.select_dtypes('category')
for col in train.columns:
    if col in cat_cols:
        train[col] = train[col].cat.codes

In [ ]:
%%time
train.to_pickle('input/train_v3.pkl')